# Basic agentic workflows, for a marketing agent use case

This notebook walks through 3 simple multi-LLM workflows, with the use case of a marketing agent being the guiding example throughout.

1. **Prompt-chaining**: Decompose a task into sequential subtasks, where eaech step builds on previous results.
2. **Parallelization**: Distributes independent subtasks across multiple LLMs or concurrent processing.
3. **Routing**: Dynamically selects specialized LLM paths based on input characteristics.

Our motivating example will be creating a restaurant marketing AI agent that writes marketing copy across various mediums (email blast, blog post, social media). We'll see how each of these methods approaches this task.

#### Step 0: some setup

In [14]:
import sys
import os

project_root = os.path.abspath(os.path.dirname(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("PYTHONPATH set to:", project_root)

PYTHONPATH set to: /Users/mark/Documents/work/demos/ai_agent_learning_examples


In [22]:
import sys
print("Python executable:", sys.executable)
print("\nPython path:")
for i, path in enumerate(sys.path):
    if i >= 1:
        break
    print(f"  {path}")

print(f"\nCurrent working directory: {os.getcwd()}")
print(f"Directory contents:")
import os
for i, item in enumerate(os.listdir('.')):
    if i >= 1:
        break
    print(f"  {i}: {item}")


Python executable: /Users/mark/anaconda3/envs/ai_agent_learning_examples/bin/python

Python path:
  /Users/mark/Documents/work/demos/ai_agent_learning_examples

Current working directory: /Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples
Directory contents:
  0: models.py


## 1. Prompt-chaining

This method breaks down a task into a series of sequential subtasks, where each step builds on previous results.

For our example, we can break down the marketing AI agent chain into each of these steps:

1. Create the idea to promote.
2. Evaluate and score the ideas. Then pick one to continue with.
3. Expand the idea into a blog post.
4. Score/evaluate the blog post.
5. Summarize blog post for email.
6. Score/evaluate the email-formatted result.
7. Create social media captions.
8. Score/evaluate the social media captions.

Let's do each of these in turn.

### 1a. Idea generation

Here, let's create a first pass of our prompt for our idea generation step.

In [10]:
idea_generation_prompt = """
You are a marketing agent. You are given a product and a target audience.
You will generate a list of ideas for a marketing campaign.

Client: American-style brunch restaurant.

Product: Brunch menu. Things we want to promote include:
- $9.99 full breakfast with coffee. Features 2 eggs, 3 pieces of bacon
(or choice of meat), potatoes, fresh fruit, and coffee.
- Hearty Mediterranean wrap for lunch
- Popular $20.99 Sunday brunch buffet featuring full breakfast options
(eggs, pancakes, waffles, bacon, sausage, hash browns, omelettes)
and lunch options (salad bar, meats, etc.).

Target audience:
    - Families
    - Health-minded individuals
    - Brunch group gatherings
"""

Let's actually pass this into an LLM to improve the prompt. We can use an LLM for this prompt improving task, and there are explicit tools like Claude's [prompt improver](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/prompt-improver) made for this task alone.

Here's the improved version that ChatGPT gave me:


In [11]:
idea_generation_prompt = """
You are an expert marketing agent helping a neighborhood American-style brunch restaurant design a creative and targeted marketing campaign. You will be given product offerings and a target audience. Your job is to generate 5–7 campaign ideas that highlight the value of the offerings and appeal to the specific interests of the audience segments.

Client: Independent American-style brunch restaurant known for quality and community.

Product offerings to highlight:
1. Budget-friendly $9.99 full breakfast combo — includes 2 eggs, choice of meat (bacon, sausage, or veggie patty), breakfast potatoes, fresh fruit, and coffee.
2. Hearty and nutritious Mediterranean wrap — packed with veggies and lean protein, great for health-conscious diners.
3. Popular $20.99 all-you-can-eat Sunday brunch buffet — includes full breakfast classics (eggs, pancakes, waffles, bacon, sausage, hash browns, omelets) and lunch options (salad bar, seasonal meats, and sides).

Target audience:
- Families looking for weekend outings
- Health-conscious individuals seeking nutritious, flavorful options
- Social brunch groups celebrating milestones or gathering casually

Output:
- A list of 5-7 specific marketing campaign ideas
- Each idea should include a title and a 2-3 sentence explanation
- Tailor each idea to one or more of the audience segments
- Highlight value, experience, or emotional appeal
"""

Let's also make sure that our prompt conforms to a specific format.


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage


In [17]:
from marketing_agent_examples.models import ProposedIdeasWrapper

parser = PydanticOutputParser(pydantic_object=ProposedIdeasWrapper)

In [23]:
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
        {idea_generation_prompt}
        {format_instructions}
    """,
    input_variables=[],
    partial_variables={
        "format_instructions": format_instructions,
        "idea_generation_prompt": idea_generation_prompt
    }
)

In [24]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

/var/folders/9g/nhh_jmv17sd2604y43zdctzw0000gn/T/ipykernel_93715/940076340.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)


In [25]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=prompt.format())
]

Let's try this out and see what we get.

In [26]:
response = llm(messages)
parsed = parser.parse(response.content)

/var/folders/9g/nhh_jmv17sd2604y43zdctzw0000gn/T/ipykernel_93715/3050779672.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


In [28]:
for i, idea in enumerate(parsed.ideas, 1):
    print(f"\n=== Idea {i} ===")
    print(f"Idea: {idea.idea}")
    print(f"Audience: {idea.audience}")
    print(f"Campaign Message: {idea.campaign_message}")
    print("=" * 50)


=== Idea 1 ===
Idea: Family Fun Brunch Day
Audience: Families looking for weekend outings
Campaign Message: Create lasting memories with our budget-friendly $9.99 full breakfast combo, perfect for families!

=== Idea 2 ===
Idea: Healthy Start Sundays
Audience: Health-conscious individuals seeking nutritious, flavorful options
Campaign Message: Kickstart your week with our nutritious Mediterranean wrap, packed with fresh veggies and lean protein!

=== Idea 3 ===
Idea: Brunch & Celebrate
Audience: Social brunch groups celebrating milestones or gathering casually
Campaign Message: Celebrate life’s moments with our all-you-can-eat Sunday brunch buffet, perfect for gatherings!

=== Idea 4 ===
Idea: Brunch Buddy Loyalty Program
Audience: Families looking for weekend outings and social brunch groups
Campaign Message: Join our Brunch Buddy Loyalty Program and earn rewards for every visit!

=== Idea 5 ===
Idea: Weekend Brunch Specials
Audience: Families looking for weekend outings and health-c

### 1b. Idea evaluation

### 1e. Chaining it all together.

Let's now chain all of these responses together.

## 2. Parallelization

(take the steps from above and pick some to rewrite or redo to include parallelization, and why to do parallelization in the first place. Should explain why parallelization is appropriate in some cases).

## 3. Routing

(take the steps from above and pick some to rewrite or redo to include routing, and why to do routing in the first place. Should explain why routing is appropriate in some cases).

## 4. Orchestrator-subagents

## 5. Evaluator-optimizer

## What's next?
(suggestions for how to improve what we've built, e.g., evals, how to optimize, etc.)

(could put this in a different Jupyter notebook TBH? Something about follow-ups? IDK if best here, since this would likely be generic across all workflows)